In [37]:
%%time
#Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pandas.plotting import autocorrelation_plot
import pmdarima as pm
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import os

os.getcwd()

Wall time: 0 ns


'C:\\Users\\marin\\Documents\\Controlador PID\\Bases'

## Abrir la BM y filtrar sólo por salidas a partir de enero del 2018

In [38]:
%%time
#Tomar datos que son desde el 2018
BM  = pd.read_csv('BM.csv')
lista = pd.read_csv('articulos_experimento.csv')
lista = lista["0"].to_numpy()


BM["Fecha"]=pd.to_datetime(BM.Fecha)
BM = BM[BM["Fecha"]>="2018-01-01"]
BM['Semana'] = pd.to_datetime(BM.Fecha.dt.week.astype(str)+BM.Fecha.dt.year.astype(str).add('-1') ,format='%V%G-%u')
BM['Cantidad'] = pd.to_numeric(BM['Cantidad'], errors='coerce')
BM["Entrada_Salida"]  = pd.to_numeric(BM["Entrada_Salida"], errors="coerce")
BM= BM[BM["Entrada_Salida"]==1]

Wall time: 5.88 s


In [39]:
len(BM)

1670576

In [40]:
%%time
data_weekly_grouped = BM.reset_index() \
                          .groupby('IdArticulo') \
                          .resample('W-MON', on='Fecha', label='left',closed='left')
data_weekly = pd.DataFrame(data_weekly_grouped.sum()['Cantidad']).reset_index().set_index('IdArticulo')
data_weekly = data_weekly.reset_index()
data_weekly["Semana"]=data_weekly["Fecha"]

BM = data_weekly

Wall time: 14.7 s


### Contar el número de semanas que han transcurrido desde Enero del 2018

In [41]:
%%time
semanas=len(BM.groupby("Semana").count())
print("Han Transcurrido:",semanas,"Semanas desde Enero del 2018")

Han Transcurrido: 148 Semanas desde Enero del 2018
Wall time: 15 ms


## Lista de los artículos que se encuentran en la base

In [42]:
%%time

prueba= lista[:2]
prueba

Wall time: 0 ns


array([700008., 700012.])

In [43]:
%%time
#Loop para hacer predicciones con MM

IdArticulo =[]
ms4        =[]
ms8        =[]
mm4        =[]
mm8        =[]
me4        =[]
me8        =[]

for i in lista:
    print(i)

    #Crear la Serie de tiempo para un articulo en específico
    Medicamento = BM[BM["IdArticulo"]==i]
    SerieTiempo = Medicamento[["Semana", "Cantidad"]].groupby("Semana").sum()
    
    
    
    SlidingWindow=SerieTiempo;
    
    #Algoritmos de predicción
    media_cuatro=np.round(SerieTiempo["Cantidad"][-4:].mean())
    media_ocho=np.round(SerieTiempo["Cantidad"][-8:].mean())
    
    SlidingWindow['MediaMovil_4']       = SlidingWindow.iloc[:,0].rolling(window=4).mean().round();
    SlidingWindow['MediaMovil_8']       = SlidingWindow.iloc[:,0].rolling(window=8).mean().round();
    SlidingWindow['MediaExponencial_4'] = SlidingWindow.iloc[:,0].ewm(com= 4, min_periods = 4,adjust=True).mean().round();
    SlidingWindow['MediaExponencial_8'] = SlidingWindow.iloc[:,0].ewm(com= 8, min_periods = 8,adjust=True).mean().round();
    
    medm4= SlidingWindow[-1:].iloc[0]["MediaMovil_4"].round()
    medm8= SlidingWindow[-1:].iloc[0]["MediaMovil_8"].round()
    mex4= SlidingWindow[-1:].iloc[0]["MediaExponencial_4"].round()
    mex8= SlidingWindow[-1:].iloc[0]["MediaExponencial_8"].round()
   
    IdArticulo.append(i)
    ms4.append(media_cuatro)
    ms8.append(media_ocho)
    mm4.append(medm4)
    mm8.append(medm8)
    me4.append(mex4)
    me8.append(mex8)
    
d={"IdArticulo":IdArticulo, "MS4":ms4, "MS8":ms8,"MM4":mm4, "MM8": mm8, "ME4": me4, "ME8":me8}
Prediccioones_Medias=pd.DataFrame(data=d)
Prediccioones_Medias

700008.0
700012.0
700024.0
700043.0
700046.0
700052.0
700072.0
700077.0
700080.0
800017.0
800031.0
800050.0
800051.0
800093.0
800103.0
800116.0
800122.0
800148.0
800164.0
800182.0
800296.0
800297.0
800317.0
800321.0
800323.0
800344.0
800353.0
800354.0
800429.0
800491.0
800604.0
800625.0
1000005.0
1000007.0
1000018.0
1000028.0
1000031.0
1000034.0
1000040.0
1000042.0
1000046.0
1000051.0
1000053.0
1000054.0
1000055.0
1000056.0
1000060.0
1000063.0
1000064.0
1000065.0
1000067.0
1000069.0
1000070.0
1000071.0
1000077.0
1000082.0
1000095.0
1000097.0
1000099.0
1000115.0
1000119.0
1000121.0
1000122.0
1000123.0
1000124.0
1000161.0
1000162.0
1000177.0
1000184.0
1000187.0
1000189.0
1000191.0
1000193.0
1000195.0
1000201.0
1000204.0
1000207.0
1000209.0
1000210.0
1000215.0
1000220.0
1000235.0
1000243.0
1000259.0
1000261.0
1000268.0
1000278.0
1000307.0
1000326.0
1000330.0
1000341.0
2500010.0
3300046.0
3300072.0
3300091.0
3300442.0
3300540.0
3300565.0
3300604.0
3300627.0
3300642.0
3300668.0
3300705.0
33

,IdArticulo,MS4,MS8,MM4,MM8,ME4,ME8
0,700008.0,28.0,35.0,28.0,35.0,30.0,32.0
1,700012.0,2.0,5.0,2.0,5.0,4.0,4.0
2,700024.0,42.0,44.0,42.0,44.0,37.0,36.0
3,700043.0,37.0,39.0,37.0,39.0,37.0,37.0
4,700046.0,6.0,3.0,6.0,3.0,4.0,3.0
...,...,...,...,...,...,...,...
224,3800066.0,156.0,148.0,156.0,148.0,147.0,141.0
225,3800067.0,1419.0,1424.0,1419.0,1424.0,1410.0,1382.0
226,3800068.0,1015.0,1075.0,1015.0,1075.0,1048.0,1046.0
227,3800069.0,554.0,542.0,554.0,542.0,538.0,535.0


In [47]:
Prediccioones_Medias.to_csv("Predicciones_Medias.csv", index=False)